In [26]:
#import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import config
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

ModuleNotFoundError: No module named 'imblearn'

In [3]:
#sqlalcehmy database connection
database = f'postgres://{config.user}:{config.db_password}@{config.host}/postgres'
engine = create_engine(database)

In [4]:
beer_db = pd.read_sql_query('''SELECT * FROM beer_data;''', con=engine)
beer_db.head()

,abv,ibu,id,name,style,brewery_id,ounces,brewery_idbrewery_,namebrewery_,city,state
0,0.061,60.0,1979,Bitter Bitch,American Pale Ale (APA),177,12.0,177,18th Street Brewery,Gary,IN
1,0.099,92.0,1036,Lower De Boom,American Barleywine,368,8.4,368,21st Amendment Brewery,San Francisco,CA
2,0.079,45.0,1024,Fireside Chat,Winter Warmer,368,12.0,368,21st Amendment Brewery,San Francisco,CA
3,0.044,42.0,876,Bitter American,American Pale Ale (APA),368,12.0,368,21st Amendment Brewery,San Francisco,CA
4,0.049,17.0,802,Hell or High Watermelon Wheat (2009),Fruit / Vegetable Beer,368,12.0,368,21st Amendment Brewery,San Francisco,CA


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(beer_db[['ibu','abv']], beer_db[['style']], random_state=48)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train

,style
1103,Bock
80,Gose
414,American IPA
844,American Stout
612,American Strong Ale
...,...
966,American Pale Ale (APA)
944,Hefeweizen
347,American IPA
1361,American IPA


In [46]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
fvals = le.fit_transform(np.ravel(beer_db[['style']]))
fvals = np.unique(fvals)
print(fvals)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89]


In [69]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100, random_state=48).fit(X_train,(np.ravel(y_train)), sample_weight=None)

print(rfc)

preds = rfc.predict(X_test)
print(preds)

RandomForestClassifier(random_state=48)
['Saison / Farmhouse Ale' 'American Pale Ale (APA)' 'American Porter'
 'Saison / Farmhouse Ale' 'Fruit / Vegetable Beer' 'American Black Ale'
 'American IPA' 'American Pale Ale (APA)' 'American Strong Ale'
 'American Pale Ale (APA)' 'American IPA' 'American Pale Ale (APA)'
 'Märzen / Oktoberfest' 'Bock' 'American Brown Ale' 'Bock' 'American IPA'
 'American Double / Imperial IPA' 'American Pale Ale (APA)' 'American IPA'
 'American Double / Imperial IPA' 'American IPA' 'Fruit / Vegetable Beer'
 'American Blonde Ale' 'American Double / Imperial IPA'
 'Fruit / Vegetable Beer' 'Dunkelweizen' 'American IPA' 'American IPA'
 'American IPA' 'American Amber / Red Ale' 'American Pale Wheat Ale'
 'American IPA' 'Hefeweizen' 'Hefeweizen' 'American IPA'
 'American Pale Ale (APA)' 'American Brown Ale' 'American Blonde Ale'
 'American IPA' 'American Pilsner' 'American Pale Ale (APA)'
 'American Pale Wheat Ale' 'American IPA' 'Fruit / Vegetable Beer'
 'American P

In [72]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]